In [ ]:
%load_ext watermark
%watermark

In [ ]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [2]:
%matplotlib
matplotlib.rcParams.update({'font.size': 25})

Using matplotlib backend: TkAgg


In [ ]:
def beta_continue(n,a,q,beta_guess): # foireux
    C1 = 0
    C2 = 0
    for k in range(n,0,-1):
        C1 = q**2/((beta_guess+2*n)**2 - a - C1)
        C2 = q**2/((beta_guess-2*n)**2 - a - C2)
        
    return a + C1 + C2

In [3]:
def beta_continue_alamano(a,q,beta_guess): # a l'air ok
    C1 = q**2/( (beta_guess+2)**2 - a - q**2/( (beta_guess+4)**2 - a - q**2/( (beta_guess+6)**2 - a - q**2/( (beta_guess+8)**2 - a - q**2/( (beta_guess+10)**2 - a ) ) ) ) )
    C2 = q**2/( (beta_guess-2)**2 - a - q**2/( (beta_guess-4)**2 - a - q**2/( (beta_guess-6)**2 - a - q**2/( (beta_guess-8)**2 - a - q**2/( (beta_guess-10)**2 - a  ) ) ) ) )
    return a + C1 + C2

In [4]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.00e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23  # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [5]:
beta_guess = 0.402
a = 0
q = 0.60
for k in range(15):
    beta_guess = beta_continue_alamano(a,q,beta_guess)
    beta_guess = sqrt(beta_guess)
#     print(beta_guess)
print('non adiab',beta_guess)
print('adiab    ',sqrt(a+q**2/2))

print('non adiab',0.5*beta_guess*Omega/1000/2/pi)
print('adiab    ',0.5*sqrt(a+q**2/2)*Omega/1000/2/pi)

non adiab 0.46224946254501875
adiab     0.4242640687119285
non adiab 462.2494625450187
adiab     424.26406871192853


In [6]:
N_ions = 1024

# for k,l in enumerate(list(points_and_coord)):
my_udc = 2
my_urf = 64.61
q = 4 * C_e * my_urf / (m_Ca*r0**2*Omega**2)
# omegaz = sqrt( 2 * C_e * my_udc / (m_Ca*d0**2) )
# omegax_2.append(Omega**2 / 4 *  q**2/2)
# beta2 = q**2/(2-q**2) - 7*q**4/(32*4) + 29*q**6/(64*4*9)
# omegax_2bis.append(0.25*Omega**2 * beta2 )
# omegar_2 = Omega**2 / 4 * ( q**2/2 - 0.5*(omegaz/Omega)**2 )

In [7]:
a_u = 0
q_u = q

In [22]:
def f_beta(a,q,beta_guess):
    return q**2/( (beta_guess+2)**2 - a - q**2/( (beta_guess+4)**2 - a - q**2/( (beta_guess+6)**2 - a - q**2/( (beta_guess+8)**2 - a - q**2/( (beta_guess+10)**2 - a ) ) ) ) )

def beta_continue_alamano(a,q,beta_guess): #a l'air ok
    return a + f_beta(a,q,beta_guess) + f_beta(a,q,-beta_guess)

In [8]:
beta_guess = 0.402
a = 0
q = 0.60
for k in range(15):
    beta_guess = beta_continue_alamano(a,q,beta_guess)
    beta_guess = sqrt(beta_guess)
#     print(beta_guess)
print('non adiab',beta_guess)
print('adiab    ',sqrt(a+q**2/2))

print('non adiab',0.5*beta_guess*Omega/1000/2/pi)
print('adiab    ',0.5*sqrt(a+q**2/2)*Omega/1000/2/pi)

non adiab 0.46224946254501875
adiab     0.4242640687119285
non adiab 462.2494625450187
adiab     424.26406871192853


In [9]:
# beta adiabatique
q = linspace(0,1,10)
beta_adia = sqrt(a+q**2/2)

In [18]:
# beta non adiabatique
a = 0
q = linspace(0,1,11)
beta_adia = sqrt(a+q**2/2)
beta_nonadia = np.zeros((len(beta_adia)))
print('  q ', '    beta non-a', ' beta a')
for j,k in enumerate(q):
    for l in range(15):
        beta_nonadia[j] = beta_continue_alamano(a,k,beta_nonadia[j])
        beta_nonadia[j] = sqrt(beta_nonadia[j])
    print('>',f'{k:.02f}','->',f'{beta_nonadia[j]:.03f}',' ~ ',f'{beta_adia[j]:.03f}')
print('')

print('|    q   |     Urf     |    beta    |      fx      |     Udc     |')
print('|:--------:|:-------------:|:------------:|:--------------:|:-------------:|')
omega_nonadia = np.zeros((len(beta_adia)))    
Udc_for_sphere = np.zeros((len(beta_adia)))
for i,j in enumerate(beta_nonadia):
    omega_nonadia[i]  = 0.5*Omega*j
    Udc_for_sphere[i] = m_Ca*d0**2/(2*kappa_simion*C_e)*omega_nonadia[i]**2
    Urf = q[i]*m_Ca*r0**2*Omega**2/(4*C_e)
    print(f'| {q[i]:.02f} | {Urf:7.03f} | {j:.03f} | {omega_nonadia[i]/2/np.pi:10.03f} | {Udc_for_sphere[i]:07.03f} |')

  q      beta non-a  beta a
> 0.00 -> 0.000  ~  0.000
> 0.10 -> 0.071  ~  0.071
> 0.20 -> 0.143  ~  0.141
> 0.30 -> 0.216  ~  0.212
> 0.40 -> 0.293  ~  0.283
> 0.50 -> 0.374  ~  0.354
> 0.60 -> 0.462  ~  0.424
> 0.70 -> 0.563  ~  0.495
> 0.80 -> 0.689  ~  0.566
> 0.90 -> 0.912  ~  0.636
> 1.00 -> nan  ~  0.707

|    q   |     Urf     |    beta    |      fx      |     Udc     |
|:--------:|:-------------:|:------------:|:--------------:|:-------------:|
| 0.00 |   0.000 | 0.000 |      0.000 | 000.000 |
| 0.10 |  10.250 | 0.071 |  70849.553 | 000.609 |
| 0.20 |  20.500 | 0.143 | 142551.327 | 002.464 |
| 0.30 |  30.751 | 0.216 | 216059.135 | 005.661 |
| 0.40 |  41.001 | 0.293 | 292566.211 | 010.380 |
| 0.50 |  51.251 | 0.374 | 373744.122 | 016.940 |
| 0.60 |  61.501 | 0.462 | 462249.463 | 025.913 |
| 0.70 |  71.751 | 0.563 | 563066.161 | 038.449 |
| 0.80 |  82.002 | 0.689 | 688563.739 | 057.498 |
| 0.90 |  92.252 | 0.912 | 911952.404 | 100.857 |
| 1.00 | 102.502 | nan |        nan | 0000n

/tmp/ipykernel_9226/1436104007.py:10: RuntimeWarning: invalid value encountered in sqrt
  beta_nonadia[j] = sqrt(beta_nonadia[j])


In [28]:
a = 0.02
q = linspace(0,0.908,1000)
beta_adia = sqrt(a+q**2/2)
beta_nonadia = np.zeros((len(beta_adia)))

for j,k in enumerate(q):
    for l in range(750):
        beta_nonadia[j] = beta_continue_alamano(a,k,beta_nonadia[j])
        beta_nonadia[j] = sqrt(beta_nonadia[j])

fname = 'compare_betas'
fig = plt.figure(fname,clear=True)
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
# title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

ax1.plot(q,beta_adia,ls='--',label=r'$\beta_a$')
ax1.plot(q,beta_nonadia,ls=':',label=r'$\beta$')

ax1.set_xlabel('$q_u$')
# ax1.set_ylabel(r'$\beta$')
ax1.grid()
ax1.legend()
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.2))
ax1.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.2))

ax2.plot(q,beta_nonadia/beta_adia,label=r'$\beta/\beta_a$',color='xkcd:rouge')

ax2.set_xlabel('$q_u$')
# ax1.set_ylabel(r'$\beta$')
ax2.grid()
ax2.legend()
ax2.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.2))
ax2.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))

fig.set_size_inches(14,6)

fig.tight_layout()
# subplots_adjust(hspace=0.05, bottom=0.163, top = 0.94,left=0.10,right=1.02)
savefig(fname+'.png',dpi=300)

KeyboardInterrupt: 